In [1]:
import heapq
from collections import defaultdict, Counter
import numpy as np
from PIL import Image

# Node class for Huffman Tree
class Node:
    def __init__(self, freq, symbol, left=None, right=None):
        self.freq = freq
        self.symbol = symbol
        self.left = left
        self.right = right
        self.huff = ''

    def __lt__(self, nxt):
        return self.freq < nxt.freq

# Function to build the Huffman Tree
def build_huffman_tree(data):
    frequency = Counter(data)
    heap = [Node(freq, symbol) for symbol, freq in frequency.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        left.huff = '0'
        right.huff = '1'
        newNode = Node(left.freq + right.freq, left.symbol + right.symbol, left, right)
        heapq.heappush(heap, newNode)

    return heap[0]

# Function to generate Huffman codes
def generate_codes(node, val=''):
    codes = {}
    newVal = val + node.huff

    if node.left:
        codes.update(generate_codes(node.left, newVal))
    if node.right:
        codes.update(generate_codes(node.right, newVal))

    if not node.left and not node.right:
        codes[node.symbol] = newVal

    return codes

# Function to compress the image
def compress_image(image_path):
    image = Image.open(image_path).convert('L')
    data = np.array(image).flatten()
    huffman_tree = build_huffman_tree(data)
    huffman_codes = generate_codes(huffman_tree)

    compressed_data = ''.join([huffman_codes[pixel] for pixel in data])
    return compressed_data, huffman_tree

# Function to decompress the image
def decompress_image(compressed_data, huffman_tree, shape):
    current_node = huffman_tree
    decompressed_data = []

    for bit in compressed_data:
        if bit == '0':
            current_node = current_node.left
        else:
            current_node = current_node.right

        if not current_node.left and not current_node.right:
            decompressed_data.append(current_node.symbol)
            current_node = huffman_tree

    decompressed_image = np.array(decompressed_data).reshape(shape)
    return decompressed_image

# Example usage
image_path = 'cap-america.jpg'
compressed_data, huffman_tree = compress_image(image_path)
image = Image.open(image_path).convert('L')
shape = np.array(image).shape

decompressed_image = decompress_image(compressed_data, huffman_tree, shape)
decompressed_image = Image.fromarray(decompressed_image)
decompressed_image.show()


C:\Users\ASUS\AppData\Local\Temp/ipykernel_22632/2088874688.py:29: RuntimeWarning: overflow encountered in ubyte_scalars
  newNode = Node(left.freq + right.freq, left.symbol + right.symbol, left, right)


In [ ]:
from PIL import Image
import numpy as np

# Function to perform LZW compression
def lzw_compress(uncompressed):
    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): i for i in range(dict_size)}
    w = ""
    compressed_data = []

    for c in uncompressed:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            compressed_data.append(dictionary[w])
            dictionary[wc] = dict_size
            dict_size += 1
            w = c

    if w:
        compressed_data.append(dictionary[w])
    return compressed_data

# Function to perform LZW decompression
def lzw_decompress(compressed):
    # Build the dictionary.
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}
    w = chr(compressed.pop(0))
    decompressed_data = [w]

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError('Bad compressed k: %s' % k)
        decompressed_data.append(entry)

        # Add w+entry[0] to the dictionary.
        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry
    return ''.join(decompressed_data)

# Function to compress an image
def compress_image(image_path):
    image = Image.open(image_path).convert('L')
    data = np.array(image).flatten()
    data_str = ''.join([chr(pixel) for pixel in data])
    compressed_data = lzw_compress(data_str)
    return compressed_data, image.size

# Function to decompress an image
def decompress_image(compressed_data, image_size):
    decompressed_data_str = lzw_decompress(compressed_data)
    decompressed_data = [ord(char) for char in decompressed_data_str]
    decompressed_image = np.array(decompressed_data).reshape(image_size)
    return decompressed_image

# Example usage
image_path = 'lzw.png'
compressed_data, image_size = compress_image(image_path)
decompressed_image = decompress_image(compressed_data, image_size)
decompressed_image = Image.fromarray(decompressed_image)
decompressed_image.show()


In [2]:
from skimage.metrics import structural_similarity as ssim
import cv2

def calculate_ssim(original, compressed):
    ssim_value, _ = ssim(original, compressed, full=True)
    return ssim_value

# Example usage
original = cv2.imread('lzw.png', cv2.IMREAD_GRAYSCALE)
compressed = cv2.imread('tmpwgm96wro.PNG', cv2.IMREAD_GRAYSCALE)
ssim_value = calculate_ssim(original, compressed)
print(f'SSIM: {ssim_value}')


SSIM: 0.1805045287842346


In [ ]:
from skimage import io, img_as_float
from skimage.metrics import structural_similarity as ssim
import numpy as np

# Function to calculate SSIM between two images
def calculate_ssim(image1_path, image2_path):
    # Read the images
    image1 = img_as_float(io.imread(image1_path))
    image2 = img_as_float(io.imread(image2_path))

    # Ensure the images have the same dimensions
    if image1.shape != image2.shape:
        raise ValueError("Input images must have the same dimensions.")

    # Calculate SSIM
    ssim_value, ssim_image = ssim(image1, image2, full=True)
    return ssim_value, ssim_image

# Example usage
image1_path = 'lwx.png'
image2_path = 'tmpwgm96wro.png'
ssim_value, ssim_image = calculate_ssim(image1_path, image2_path)

print(f"SSIM: {ssim_value}")

# Save the SSIM image for visualization
io.imsave('ssim_image.png', ssim_image)


In [3]:
def lzw_compress(uncompressed):
    """Compress a string to a list of output symbols."""
    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): i for i in range(dict_size)}
    w = ""
    result = []
    for c in uncompressed:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            # Add wc to the dictionary.
            dictionary[wc] = dict_size
            dict_size += 1
            w = c

    # Output the code for w.
    if w:
        result.append(dictionary[w])
    return result

def read_png(file_path):
    with open(file_path, 'rb') as file:
        return file.read()

def write_compressed(file_path, compressed_data):
    with open(file_path, 'wb') as file:
        for data in compressed_data:
            file.write(data.to_bytes(4, byteorder='big'))

# Example usage:
png_data = read_png('lzw.png')
compressed_data = lzw_compress(png_data.decode('latin1'))
write_compressed('compressed.lzw', compressed_data)


In [6]:
import zlib

def deflate_compress(data):
    """Compress data using DEFLATE algorithm."""
    return zlib.compress(data)

def read_png(file_path):
    with open(file_path, 'rb') as file:
        return file.read()

def write_file(file_path, data):
    with open(file_path, 'wb') as file:
        file.write(data)

# Example usage:
# Read the PNG file
png_data = read_png('lzw.png')
# Compress the PNG data
compressed_data = deflate_compress(png_data)
write_file('compressed.deflate', compressed_data)

print("Compression completed successfully.")


Compression completed successfully.


In [7]:
import zlib

def deflate_decompress(data):
    """Decompress data using DEFLATE algorithm."""
    return zlib.decompress(data)

def read_file(file_path):
    with open(file_path, 'rb') as file:
        return file.read()

def write_png(file_path, data):
    with open(file_path, 'wb') as file:
        file.write(data)

# Example usage:
# Read the compressed file
compressed_data = read_file('compressed.deflate')

# Decompress the PNG data
decompressed_data = deflate_decompress(compressed_data)
write_png('decompressed.png', decompressed_data)

print("Decompression completed successfully.")


Decompression completed successfully.
